In [1]:
# Notebook header
import os, imolecule
import numpy as np
from pybat.core import Cathode
from pymatgen import Composition, Element, Specie
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Honeycomb structures - Sn-rich phase

In this notebook we will set up a set of configurations for the Li1.2Sn0.8O2 phase, derived from the Li2SnO3 structure.


> **Note**: As the code below relies on my own pre-alpha code ([pybat](https://github.com/mbercx/pybat)), it is possible that the code has changed since this jupyter notebook was set up. Please contact me via [researchgate](https://www.researchgate.net/profile/Marnik_Bercx) if you run into any issues, or have questions.

---

We'll start from a primitive unit cell of Li$_2$SnO$_3$:

In [2]:
cat = Cathode.from_file("Li2SnO3_icsd_prim.cif")
cat.add_site_property("magmom", [0]*24)
print(cat)

Full Formula (Li8 Sn4 O12)
Reduced Formula: Li2SnO3
abc   :   5.300540   5.300540  10.032000
angles:  84.960116  95.039884  59.930868
Sites (24)
  #    #VESTA  SP         a       b       c    magmom
---  --------  ----  ------  ------  ------  --------
  0         1  Li    0.683   0.161   0.001          0
  1         2  Li    0.317   0.839   0.999          0
  2         3  Li    0.161   0.683   0.499          0
  3         4  Li    0.839   0.317   0.501          0
  4         5  Li    0.5     0       0.5            0
  5         6  Li    0       0.5     0              0
  6         7  Li    0.917   0.917   0.75           0
  7         8  Li    0.083   0.083   0.25           0
  8         9  Sn    0.5835  0.5835  0.75           0
  9        10  Sn    0.4165  0.4165  0.25           0
 10        11  Sn    0.2492  0.2492  0.75           0
 11        12  Sn    0.7508  0.7508  0.25           0
 12        13  O     0.6066  0.874   0.8667         0
 13        14  O     0.3934  0.126   0.1333  

Let's check if this structure has the required space group:

In [3]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

SpacegroupAnalyzer(cat).get_space_group_symbol()

'C2/c'

Perfect, this corresponds to the space group described in [Sathiya et al.](https://www.nature.com/articles/nmat3699). Next we need to set up a suitable supercell that has 24 unit cells of the base structure and allows for honeycomb ordering. For this unit cell, a 2x2x1 supercell should do the trick. However, we need to distinguish the Li sites in the Sn-layer from the ones in the Li-layer, because the former can also contain Sn instead of Li. We'll keep track of these by substituting them with a placeholder element (Lr), which will be replaced later by either Li or Sn.

In [4]:
super_cat = cat.copy()

super_cat.replace(6, "Lr", properties={"magmom": 0})
super_cat.replace(7, "Lr", properties={"magmom": 0})

super_cat.make_supercell([2,2,1])
print(super_cat)

Full Formula (Li24 Lr8 Sn16 O48)
Reduced Formula: Li3Lr(SnO3)2
abc   :  10.601079  10.601079  10.032000
angles:  84.960116  95.039884  59.930868
Sites (96)
  #    #VESTA  SP          a        b       c    magmom
---  --------  ----  -------  -------  ------  --------
  0         1  Li    0.3415   0.0805   0.001          0
  1         2  Li    0.3415   0.5805   0.001          0
  2         3  Li    0.8415   0.0805   0.001          0
  3         4  Li    0.8415   0.5805   0.001          0
  4         5  Li    0.1585   0.4195   0.999          0
  5         6  Li    0.1585   0.9195   0.999          0
  6         7  Li    0.6585   0.4195   0.999          0
  7         8  Li    0.6585   0.9195   0.999          0
  8         9  Li    0.0805   0.3415   0.499          0
  9        10  Li    0.0805   0.8415   0.499          0
 10        11  Li    0.5805   0.3415   0.499          0
 11        12  Li    0.5805   0.8415   0.499          0
 12        13  Li    0.4195   0.1585   0.501          0
 13 

In [5]:
super_cat.to("cif", "temp.cif")

That supercel looks fine! The unit cell can reproduce the desired stoichiometry, as well as the honeycomb pattern. Now, we need to generate the Li-Sn configurations.

---

In [6]:
conf_list = super_cat.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(super_cat) if s.specie == Element("Lr")],
    cation_list=["Li", "Sn"],
    sizes=[1,],
    concentration_restrictions={"Li": (0.30, 0.305)}, #
    max_configurations=1e2
)

Let's see how many structures this produces:

In [7]:
len(conf_list)

5

Only 5 possible configurations! That makes life somewhat easier, as we now only have to calculate these 5 possible configurations for the Li1.2Sn0.8O2 stoichiometry. Let's check the space groups of these various structures:

In [8]:
for s in conf_list:
    print(SpacegroupAnalyzer(s).get_space_group_symbol())

C2
P1
C2
C2
P1


---

Let's set up the workflow and submit it to the mongoDB server:

In [9]:
from pybat.workflow.workflows import get_wf_configurations
from pybat.config import load_config

lpad = load_config("launchpad", "leibniz")
base_dir = "/scratch/antwerpen/202/vsc20248/batteries"

In [10]:
wf = get_wf_configurations(
    structure=super_cat,
    directory=os.path.join(base_dir, str(conf_list[0].composition).replace(" ", "")),
    substitution_sites=[],
    sizes=[],
    configuration_list=conf_list,
    functional=("scanVdW", {}),
    in_custodian=True,
    number_nodes=4
)

In [11]:
lpad.add_wf(wf)

2020-03-03 18:52:28,685 INFO Added a workflow. id_map: {-1: 12}


{-1: 12}